# 01 - Intro
Här kommer de mest basala och banala exemplen på hur man kan använda Jupyter Notebooks för webbanalys.

**Noob på Jupyter notebooks?** 
Markera den grå cell med kod du vill köra och klicka antingen på "Run"-knappen eller använd tangentbordskommandot Ctrl + Enter. De vita cellerna är dokumentation. De skrivs med fikonspråket [Markdown](https://www.markdowntutorial.com). För att ändra de vita cellerna dubbelklickar du i dem.

## 01:00 – Vilka moduler finns i systemet?
Listar vilka moduler som finns. Detta hjälper dig få reda på vad du kan använda i den miljö du kör din notebook. Det är i denna lista du kan kolla om något saknas. 
För denna första notebook behöver du hitta följande i nedan lista:
* bs4
* requests

In [10]:
help('modules')


Please wait a moment while I gather a list of all available modules...

Crypto              bz2file             keyring             queue
Cython              cProfile            keyword             quopri
Dora                cachecontrol        kiwisolver          random
IPython             caffe2              klein               re
Kqlmagic            calendar            knack               readline
OpenSSL             certifi             lancet              reprlib
PIL                 cffi                lazy_object_proxy   requests
PyQt5               cgi                 lib2to3             requests_oauthlib
__future__          cgitb               libarchive          resource
_ast                characteristic      libfuturize         retrying
_ast27              chardet             libpasteurize       rlcompleter
_ast3               chunk               lightgbm            rmagic
_asyncio            click               line_profiler       rope
_bisect             client            

## 01:01 – Kolla statuskod och HTTP-headers med requests
*Requests* är en modul som enkelt hjälper till med förfrågningar över nätet, som att hämta innehållet från en webbsidan, posta saker, etc.

In [36]:
import requests

url = "https://www.vgregion.se"

req = requests.get(url)
print('Info om request [statuskoden]:', req.status_code)
print('Headers:\n', str(req.headers).replace(', ', ', \n'))

Info om request [statuskoden]: 200
Headers:
 {'Connection': 'Keep-Alive', 
'Transfer-Encoding': 'chunked', 
'Date': 'Mon, 
21 Oct 2019 08:31:24 GMT', 
'Content-Type': 'text/html; charset=utf-8', 
'Server': 'Microsoft-IIS/8.5', 
'Accept-Ranges': 'bytes', 
'X-AspNetMvc-Version': '5.2', 
'X-AspNet-Version': '4.0.30319', 
'X-UA-Compatible': 'IE=edge', 
'Content-Security-Policy': 'frame-ancestors *.vgregion.se *.vastarvet.se *.angeredsnarsjukhus.se *.fhsk.se *.lodosemuseum.se *.naturbruk.nu *.nusjukvarden.se *.vgrfolkhogskolor.se *.narhalsan.se *.forsviksbruk.se *.vanersborgsmuseum.se *.vitlyckemuseum.se *.sahlgrenska.se *.naturbruk.nu *.slojdochbyggnadsvard.se *.mun-h-center.se *.maritimaklustret.se *.botaniska.se *.gnm.se *.insipio.com *.esmaker.net *.samverkandesjukvard.se *.vastfastigheter.se *.fyrbodalshalsoakademi.se *.valdinararelationer.se *.utvag.se *.vardsamverkan.se *.nationellpvkonferens19.se *.utvag.com *.biobankvast.se *.t-d.se;', 
'X-Varnish': '280987874 280149614', 
'Strict-

## 01:02 - Kolla sidtitel med BeautifulSoup
*BeautifulSoup* bearbetar innehållet från webbsidor men också XML-filer, vilket är användbart för sitemaps, RSS-flöden med mera.

In [37]:
# vi redan importerat requests sedan tidigare
# variabeln 'url' är definierad i föregående block
from bs4 import BeautifulSoup

req = requests.get(url)

soup = BeautifulSoup(req.content)
print('Sidans titel:', soup.title.text)

print('Längd på sidtitel:', len(soup.title.text), 'tecken')

Sidans titel: Västra Götalandsregionen - Västra Götalandsregionen
Längd på sidtitel: 51 tecken


## 01:03 – Räkna antalet länkar
Antalet länkar är intressant av flera anledningar. Dels finns en rekommendation om att ha färre än 200 länkar just för att det blir överväldigande många att välja emellan, men också inom SEO-kretsar vill man ha koll på antalet länkar. Det gäller att hushålla med sin "länkjuice", ha lagom med utgående länkar, etc.
Här ett utkast på hur man kan räkna länkar.

In [38]:
# vi fortsätter på ovanstående BS-kod
from urllib.parse import urlparse

alla_lankar = soup.findAll('a', href=True)
print('Antal länkar:', len(alla_lankar), 'stycken')

# bearbeta URL:en vi hämtar länkar från
o = urlparse(url)
print('Domän för sidan som hämtats:', o.netloc)

i = 0
externa_lankar = []

for lank in alla_lankar:
    # ta bort hashtag på nedan rad om du vill lista alla länkar
    # print(lank['href'])
    
    if 'http' in lank['href']:
        o_lank = urlparse(lank['href'])
        # print(o_lank.netloc)
        
        if o_lank.netloc != o.netloc:
            # extern länk har hittats
            i += 1
            if o_lank.netloc not in externa_lankar:
                # lägger endast med unika domäner
                externa_lankar.append(o_lank.netloc)

if i > 0:
    print('Antal externa länkar:', i, 'stycken')
    print('\nExterna domäner som länkas:')
    for lank in externa_lankar:
        print('*', lank)

    print('\nAndel externa länkar:', round(100 * (i / len(alla_lankar)), 1), '%')
else:
    print('\nInga externa länkar.')

Antal länkar: 130 stycken
Domän för sidan som hämtats: www.vgregion.se
Antal externa länkar: 12 stycken

Externa domäner som länkas:
* spoxy3.insipio.com
* vgrfokus.se
* www.facebook.com
* twitter.com
* www.linkedin.com
* www.youtube.com

Andel externa länkar: 9.2 %
